# **USE CASE QUERIES**
---
This notebook runs the queries necessary to show adequate cleaning and support of our use case. 

### **USE CASE** 
**Which establishments, in D, have the most food inspection violations?**

* Only the failed inspections matter, therefore, a violations is only counted against the establishment when the inspection was also failed.

* Each row is a specific establishment at a unique location.

## **BEFORE CLEANING**
---

In [353]:
import sqlite3
import pandas as pd
db_dirty = sqlite3.connect('../data/dirty_data/dirty_food_inspections.db')
cur = db_dirty.cursor()

In [354]:
Q = '''
SELECT est.estName as Establishment_Name, COUNT(inspec.violations) AS amount_of_violations
  FROM (select * 
        FROM Inspections 
        where result is "Fail") AS inspec
  INNER JOIN EstablishmentInspections AS ei
  ON inspec.inspecID = ei.inspecID
  INNER JOIN Establishments AS est
  ON est.estID = ei.estID
  GROUP BY est.estID
  ORDER BY amount_of_violations DESC
'''
cur.execute(Q)
df = pd.read_sql_query(Q, db_dirty)
df.head(20)

,Establishment_Name,amount_of_violations
0,ILLINOIS SPORTSERVICE INC,19
1,IYANZE,16
2,Orly's/Jalapeno,16
3,CHINA CAFE,14
4,JIMMY G'S,14
5,McDONALD'S,13
6,Congress Pizza,13
7,Food 4 Less Midwest #552,12
8,KING FOOD MART,11
9,"THE NEW THREE HAPPINESS, INC",11


## **AFTER CLEANING**
---

In [355]:
import sqlite3
import pandas as pd
db_clean = sqlite3.connect('../data/clean_data/food_inspections.db')
cur = db_clean.cursor()

In [356]:
Q = '''
SELECT est.estName as Establishment_Name, COUNT(inspec.number) AS amount_of_violations
  FROM (select * 
        FROM Inspections 
        where result is "Fail") AS inspec
  INNER JOIN EstablishmentInspections AS ei
  ON inspec.inspecID = ei.inspecID
  INNER JOIN Establishments AS est
  ON est.estID = ei.estID
  GROUP BY est.estID
  ORDER BY amount_of_violations DESC
'''
cur.execute(Q)
df = pd.read_sql_query(Q, db_clean)
df.head(20)

,Establishment_Name,amount_of_violations
0,JIMMY G'S,145
1,ST. REST COUNTRY KITCHEN,142
2,ORLY'S/JALAPENO,136
3,IYANZE,124
4,"THE NEW THREE HAPPINESS, INC",122
5,PHOENIX RESTAURANT,121
6,ILLINOIS SPORT SERVICE INC.,108
7,TAQUERIA EL RANCHITO,104
8,MCDONALD'S,103
9,DAATA DARBAR RESTAURENT,101


In [357]:
db_dirty.close()
db_clean.close()